# Figures and Code for FRETBursts Paper

*This notebook generates the figures for the paper **FRETBursts: Open Source Burst Analysis Toolkit for Confocal Single-Molecule FRET** The sections here are named after sections of the paper.*

In [ ]:
from fretbursts import *
import phconvert as phc
print('phconvert: ', phc.__version__)

In [ ]:
sns = init_notebook(fs=13, savefig_dpi=75)

In [ ]:
sns.palplot(sns.colors)

In [ ]:
blue, red, green, purple, orange = sns.colors[:5]

In [ ]:
figure_size = (5, 4)
default_figure = lambda: plt.subplots(figsize=figure_size)
save_figures = True

In [ ]:
def savefig(filename, **kwargs):
    if not save_figures:
        return
    import os
    name, extension = os.path.splitext(os.path.basename(filename))
    dir_ = 'figures/%s/' % name
    kwargs_ = dict(dpi=150, bbox_inches='tight', 
                   frameon=True, facecolor='white', transparent=False)
    kwargs_.update(kwargs)
    plt.savefig(dir_ + name + extension, **kwargs_)

# Data Load

In [ ]:
url = 'http://files.figshare.com/2182601/0023uLRpitc_NTP_20dT_0.5GndCl.hdf5'
download_file(url, save_dir='./data')

In [ ]:
file_name = './data/' + url.split('/')[-1]
file_name

In [ ]:
import os
from glob import glob

In [ ]:
nb_dir = '/Users/anto/Google Drive/notebooks/'
data_dir = nb_dir + 'multispot/8spot_paper/data/2012-11-26/'

data_dir = os.path.abspath(data_dir) + '/'
assert os.path.exists(data_dir), "Path '%s' does not exist." % data_dir

file_list = sorted(f for f in glob(data_dir + '*.hdf5') if '_BKG' not in f)
file_list

labels = ['17d']#, '27d', '7d', '12d', '22d']
files_dict = {lab: fname for lab, fname in zip(labels, file_list)}
files_dict

In [ ]:
phc.hdf5._check_version(files_dict['17d'])

In [ ]:
dd = {k: loader.photon_hdf5(v) for k, v in files_dict.items()}
for d in dd.values():
    loader.usalex_apply_period(d)
    d.calc_bg(bg.exp_fit, time_s=30, tail_min_us='auto', F_bg=1.7)

In [ ]:
dd.keys()

In [ ]:
d = loader.photon_hdf5(file_name)

In [ ]:
#bpl.plot_alternation_hist(d)
loader.usalex_apply_period(d)

# smFRET Burst Analysis

## Background Estimation

In [ ]:
d.calc_bg(bg.exp_fit, time_s=20, tail_min_us='auto', F_bg=1.7)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(figure_size[0]*2, figure_size[1]), sharey=True)

dplot(d, hist_bg_single, tmax=20e-3, ph_sel=Ph_sel(Dex='Aem'), ax=ax[0],
      plot_style=dict(alpha=0.8),
      fit_style=dict(color='k'))
ax[0].legend(['Experim. distrib.', 'Exponential fit'])

dplot(d, hist_bg, ax=ax[1], 
      fit_style=dict(lw=3),
      plot_style=dict(label=None))
#plt.legend(bbox_to_anchor=(1.04, 0.9), loc=2, fontsize=12)
ax[0].set_title('(a)')
ax[1].set_title('(b)')
savefig('ph_delays_distrib_all.png')

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(figure_size[0]*2, figure_size[1]), sharey=True)

dplot(d, timetrace_bg, ax=ax[0])
dplot(dd['17d'], timetrace_bg, ax=ax[1])

plt.legend(['All-ph', 'DexDem', 'DexAem', 'AexAem'],
           #bbox_to_anchor=(1.04, 0.9), 
           ncol=2, loc='upper left', fontsize=12)

ax[0].legend([])
ax[0].set_ylim(0, 5)
ax[1].set_ylabel('')
ax[0].set_title('(a)')
ax[1].set_title('(b)')
savefig('background_timetrace')

# Burst Search

In [ ]:
d.burst_search()

# Burst Selection

## Select the FRET Populations

In [ ]:
ds = d.select_bursts(select_bursts.size, th1=15)
ds2 = ds.select_bursts(select_bursts.naa, th1=15)

In [ ]:
sns.palplot(sns.color_palette('RdYlBu_r', 12))

In [ ]:
for cmap in ['YlGnBu_r', 'RdYlBu_r', 'Spectral_r', 'viridis']:
    plt.figure()
    c = sns.color_palette(cmap, 12)[1]
    g = alex_jointplot(ds, cmap=cmap,
                       marginal_kws=dict(hist_bar_style=dict(facecolor=c, edgecolor='k', linewidth=0.2)));
    g.ax_joint.text(-0.1, 1.1, cmap)
    savefig

In [ ]:
cmap = 'RdYlBu_r' #'Spectral_r'
c = sns.color_palette(cmap, 12)[1]
kwargs = dict(cmap=cmap, 
              marginal_kws=dict(hist_bar_style=dict(facecolor=c, edgecolor='k', linewidth=0.2)))
alex_jointplot(ds, **kwargs)
alex_jointplot(ds2,**kwargs)

## Population Analysis

In [ ]:
E_fitter = bext.bursts_fitter(ds2, 'E')

In [ ]:
id(E_fitter), id(ds2.E_fitter)

In [ ]:
E_fitter.fit_histogram(model=mfit.factory_two_gaussians(), verbose=False)

In [ ]:
fit_res = ds2.E_fitter.fit_res[0]
print(fit_res.fit_report())

In [ ]:
fit_res.aic

In [ ]:
fit_res.bic

In [ ]:
fit_res.redchi

In [ ]:
fit_res.plot_residuals()
fit_res.plot_fit()
plt.ylim(-0.5,3);

In [ ]:
fig.axes

In [ ]:
sns.palplot(sns.colors)

In [ ]:
sns.palplot(sns.color_palette('Blues', 12))

In [ ]:
fig, ax = default_figure()

cmap = 'Blues' #'Spectral_r'
c = sns.color_palette(cmap, 12)[-5]

dplot(ds2, hist_fret, show_model=True, ax=ax, 
      hist_bar_style=dict(facecolor=c, edgecolor='k', linewidth=0.3))
ax.set_title('')
ax.set_xlim(0, 1.1)